# Importações

In [1]:
import pandas as pd
import numpy as np
import requests
import json

# Extração

## Tabela Deputados - período: 01/2024 a 12/2024 - ordem ascendente por nome

In [2]:
url = ("https://dadosabertos.camara.leg.br/api/v2/deputados?dataInicio=2024-01-01&dataFim=2024-12-31&ordem=ASC&ordenarPor=nome")

O método requests.get( ) pertence à biblioteca requests, usada para fazer requisições HTTP em Python.

Ele envia uma requisição HTTP do tipo GET para a URL fornecida (armazenada na variável url).

O método .json( ) converte a resposta recebida (resposta) de JSON para um objeto Python (geralmente um dicionário ou lista de dicionários).

In [3]:
resposta = requests.get(url)
data = resposta.json()
data.keys()

dict_keys(['dados', 'links'])

O método json.dumps( ) converte um objeto Python (como um dicionário) em uma string formatada no estilo JSON.

O parâmetro de formatação indent=4 define a identação para 4 espaços, para que o JSON não apareça em uma única linha

O parâmetro de codificação ensure_ascii=False garante que caracteres especiais, como acentos e cedilhas (ã, é, ç, etc.), sejam exibidos corretamente

Sendo assim, estou passando os parâmetros indent=4 e ensure_ascii=False para o método json.dumps( ) para melhorar a legibilidade e exibir corretamente caracteres não ASCII no JSON.

In [4]:
print(json.dumps(data, indent=4, ensure_ascii=False))

{
    "dados": [
        {
            "id": 220593,
            "uri": "https://dadosabertos.camara.leg.br/api/v2/deputados/220593",
            "nome": "Abilio Brunini",
            "siglaPartido": "PL",
            "uriPartido": "https://dadosabertos.camara.leg.br/api/v2/partidos/37906",
            "siglaUf": "MT",
            "idLegislatura": 57,
            "urlFoto": "https://www.camara.leg.br/internet/deputado/bandep/220593.jpg",
            "email": null
        },
        {
            "id": 204379,
            "uri": "https://dadosabertos.camara.leg.br/api/v2/deputados/204379",
            "nome": "Acácio Favacho",
            "siglaPartido": "MDB",
            "uriPartido": "https://dadosabertos.camara.leg.br/api/v2/partidos/36899",
            "siglaUf": "AP",
            "idLegislatura": 57,
            "urlFoto": "https://www.camara.leg.br/internet/deputado/bandep/204379.jpg",
            "email": "dep.acaciofavacho@camara.leg.br"
        },
        {
            "id": 2

In [5]:
deputados = data["dados"]  # Pegando apenas a lista de deputados

# Criando o DataFrame corretamente
df = pd.DataFrame(deputados)

In [6]:
df.head()

,id,uri,nome,siglaPartido,uriPartido,siglaUf,idLegislatura,urlFoto,email
0,220593,https://dadosabertos.camara.leg.br/api/v2/depu...,Abilio Brunini,PL,https://dadosabertos.camara.leg.br/api/v2/part...,MT,57,https://www.camara.leg.br/internet/deputado/ba...,None
1,204379,https://dadosabertos.camara.leg.br/api/v2/depu...,Acácio Favacho,MDB,https://dadosabertos.camara.leg.br/api/v2/part...,AP,57,https://www.camara.leg.br/internet/deputado/ba...,dep.acaciofavacho@camara.leg.br
2,220714,https://dadosabertos.camara.leg.br/api/v2/depu...,Adail Filho,REPUBLICANOS,https://dadosabertos.camara.leg.br/api/v2/part...,AM,57,https://www.camara.leg.br/internet/deputado/ba...,dep.adailfilho@camara.leg.br
3,221328,https://dadosabertos.camara.leg.br/api/v2/depu...,Adilson Barroso,PL,https://dadosabertos.camara.leg.br/api/v2/part...,SP,57,https://www.camara.leg.br/internet/deputado/ba...,dep.adilsonbarroso@camara.leg.br
4,204560,https://dadosabertos.camara.leg.br/api/v2/depu...,Adolfo Viana,PSDB,https://dadosabertos.camara.leg.br/api/v2/part...,BA,57,https://www.camara.leg.br/internet/deputado/ba...,dep.adolfoviana@camara.leg.br


In [7]:
df.shape

(602, 9)

In [8]:
df.columns

Index(['id', 'uri', 'nome', 'siglaPartido', 'uriPartido', 'siglaUf',
       'idLegislatura', 'urlFoto', 'email'],
      dtype='object')

In [9]:
url_detalhe = "https://dadosabertos.camara.leg.br/api/v2/deputados/220593"
resposta = requests.get(url_detalhe)
dados = resposta.json()
dados.keys()

dict_keys(['dados', 'links'])

In [10]:
print(json.dumps(dados, indent=4, ensure_ascii=False))

{
    "dados": {
        "id": 220593,
        "uri": "https://dadosabertos.camara.leg.br/api/v2/deputados/220593",
        "nomeCivil": "ABILIO JACQUES BRUNINI MOUMER",
        "ultimoStatus": {
            "id": 220593,
            "uri": "https://dadosabertos.camara.leg.br/api/v2/deputados/220593",
            "nome": "Abilio Brunini",
            "siglaPartido": "PL",
            "uriPartido": null,
            "siglaUf": "MT",
            "idLegislatura": 57,
            "urlFoto": "https://www.camara.leg.br/internet/deputado/bandep/220593.jpg",
            "email": null,
            "data": "2025-01-01",
            "nomeEleitoral": "Abilio Brunini",
            "gabinete": {
                "nome": null,
                "predio": null,
                "sala": null,
                "andar": null,
                "telefone": null,
                "email": null
            },
            "situacao": "Vacância",
            "condicaoEleitoral": "Titular",
            "descricaoStatu

In [11]:
deputados_detalhe = []    #cria um dataframe para armazenar os detalhes de cada deputado

for id in df.id:     #para cada id em df campo id
    url_detalhe = f"https://dadosabertos.camara.leg.br/api/v2/deputados/{id}"    #a url vai mudando e recebendo cada id
    resposta = requests.get(url_detalhe)    #o request faz uma busca na url anterior e retorna as informações  

    if resposta.status_code == 200:
        json_data = resposta.json()
        
        # Verifica se a chave 'dados' existe na resposta
        if 'dados' in json_data:
            dep_detalhes = pd.DataFrame([json_data["dados"]])  #coloca a resposta dentro de um dataframe - apenas o campo dados
            deputados_detalhe.append(dep_detalhes)   #adiciona esse dataframe novo ao dataframe criado lá no início
        else:
            print(f"Deputado {id} sem chave 'dados'. JSON retornado: {json_data}")
    else:
        print(f"Erro ao buscar dados do deputado {id}: Status {resposta.status_code}")

In [12]:
len(deputados_detalhe)

602

In [13]:
df_deputados = pd.concat(deputados_detalhe, ignore_index=True)

In [14]:
df_deputados.head()

,id,uri,nomeCivil,ultimoStatus,cpf,sexo,urlWebsite,redeSocial,dataNascimento,dataFalecimento,ufNascimento,municipioNascimento,escolaridade
0,220593,https://dadosabertos.camara.leg.br/api/v2/depu...,ABILIO JACQUES BRUNINI MOUMER,"{'id': 220593, 'uri': 'https://dadosabertos.ca...",99770962104,M,None,[],1984-01-31,None,MT,Cuiabá,Superior
1,204379,https://dadosabertos.camara.leg.br/api/v2/depu...,ACÁCIO DA SILVA FAVACHO NETO,"{'id': 204379, 'uri': 'https://dadosabertos.ca...",74287028287,M,None,"[https://twitter.com/acaciofavacho, https://ww...",1983-09-28,None,AP,Macapá,Superior
2,220714,https://dadosabertos.camara.leg.br/api/v2/depu...,ADAIL JOSÉ FIGUEIREDO PINHEIRO,"{'id': 220714, 'uri': 'https://dadosabertos.ca...",77267796249,M,None,[https://twitter.com/adailfilhoam?s=21&t=O_eoT...,1992-02-16,None,AM,Manaus,Superior Incompleto
3,221328,https://dadosabertos.camara.leg.br/api/v2/depu...,ADILSON BARROSO OLIVEIRA,"{'id': 221328, 'uri': 'https://dadosabertos.ca...",05585378805,M,None,[],1964-06-14,None,MG,Minas Novas,Superior
4,204560,https://dadosabertos.camara.leg.br/api/v2/depu...,ADOLFO VIANA DE CASTRO NETO,"{'id': 204560, 'uri': 'https://dadosabertos.ca...",80123848504,M,None,[],1981-02-02,None,BA,Salvador,Superior


# Transformação

### Valores ausentes

In [15]:
df_deputados.isnull().sum()

id                       0
uri                      0
nomeCivil                0
ultimoStatus             0
cpf                      0
sexo                     0
urlWebsite             601
redeSocial               0
dataNascimento           0
dataFalecimento        601
ufNascimento             1
municipioNascimento      0
escolaridade            36
dtype: int64

In [16]:
df_deputados[df_deputados['ufNascimento'].isna()]

,id,uri,nomeCivil,ultimoStatus,cpf,sexo,urlWebsite,redeSocial,dataNascimento,dataFalecimento,ufNascimento,municipioNascimento,escolaridade
501,225386,https://dadosabertos.camara.leg.br/api/v2/depu...,RENAN FERREIRINHA CARNEIRO,"{'id': 225386, 'uri': 'https://dadosabertos.ca...",13698925788,M,None,[],1993-10-23,None,None,Niterói,Superior


In [17]:
df_deputados.loc[(df_deputados['ufNascimento'].isna()), 'ufNascimento'] = "RJ"

In [18]:
df_deputados.isnull().sum()

id                       0
uri                      0
nomeCivil                0
ultimoStatus             0
cpf                      0
sexo                     0
urlWebsite             601
redeSocial               0
dataNascimento           0
dataFalecimento        601
ufNascimento             0
municipioNascimento      0
escolaridade            36
dtype: int64

In [19]:
df_deputados['escolaridade'].value_counts(dropna=False)

escolaridade
Superior                   323
Pós-Graduação               79
Superior Incompleto         47
Mestrado                    40
None                        36
Ensino Médio                31
Doutorado                   13
Ensino Fundamental          13
Mestrado Incompleto          8
Doutorado Incompleto         6
Ensino Médio Incompleto      3
Secundário                   2
Primário Incompleto          1
Name: count, dtype: int64

In [20]:
df_deputados['escolaridade'] = df_deputados['escolaridade'].fillna("Não informado")

In [21]:
df_deputados.isnull().sum()

id                       0
uri                      0
nomeCivil                0
ultimoStatus             0
cpf                      0
sexo                     0
urlWebsite             601
redeSocial               0
dataNascimento           0
dataFalecimento        601
ufNascimento             0
municipioNascimento      0
escolaridade             0
dtype: int64

In [22]:
df_deputados.drop(columns=['ultimoStatus', 'redeSocial'], inplace=True)

In [23]:
df_deputados.isnull().sum()

id                       0
uri                      0
nomeCivil                0
cpf                      0
sexo                     0
urlWebsite             601
dataNascimento           0
dataFalecimento        601
ufNascimento             0
municipioNascimento      0
escolaridade             0
dtype: int64

In [24]:
df_deputados.drop(columns=['urlWebsite', 'dataFalecimento'], inplace=True)
df_deputados.isnull().sum()

id                     0
uri                    0
nomeCivil              0
cpf                    0
sexo                   0
dataNascimento         0
ufNascimento           0
municipioNascimento    0
escolaridade           0
dtype: int64

In [25]:
df_deputados.head()

,id,uri,nomeCivil,cpf,sexo,dataNascimento,ufNascimento,municipioNascimento,escolaridade
0,220593,https://dadosabertos.camara.leg.br/api/v2/depu...,ABILIO JACQUES BRUNINI MOUMER,99770962104,M,1984-01-31,MT,Cuiabá,Superior
1,204379,https://dadosabertos.camara.leg.br/api/v2/depu...,ACÁCIO DA SILVA FAVACHO NETO,74287028287,M,1983-09-28,AP,Macapá,Superior
2,220714,https://dadosabertos.camara.leg.br/api/v2/depu...,ADAIL JOSÉ FIGUEIREDO PINHEIRO,77267796249,M,1992-02-16,AM,Manaus,Superior Incompleto
3,221328,https://dadosabertos.camara.leg.br/api/v2/depu...,ADILSON BARROSO OLIVEIRA,05585378805,M,1964-06-14,MG,Minas Novas,Superior
4,204560,https://dadosabertos.camara.leg.br/api/v2/depu...,ADOLFO VIANA DE CASTRO NETO,80123848504,M,1981-02-02,BA,Salvador,Superior


In [26]:
df_deputados['dataNascimento'] = pd.to_datetime(df_deputados['dataNascimento']).dt.strftime('%d/%m/%Y')

In [27]:
df_deputados.head()

,id,uri,nomeCivil,cpf,sexo,dataNascimento,ufNascimento,municipioNascimento,escolaridade
0,220593,https://dadosabertos.camara.leg.br/api/v2/depu...,ABILIO JACQUES BRUNINI MOUMER,99770962104,M,31/01/1984,MT,Cuiabá,Superior
1,204379,https://dadosabertos.camara.leg.br/api/v2/depu...,ACÁCIO DA SILVA FAVACHO NETO,74287028287,M,28/09/1983,AP,Macapá,Superior
2,220714,https://dadosabertos.camara.leg.br/api/v2/depu...,ADAIL JOSÉ FIGUEIREDO PINHEIRO,77267796249,M,16/02/1992,AM,Manaus,Superior Incompleto
3,221328,https://dadosabertos.camara.leg.br/api/v2/depu...,ADILSON BARROSO OLIVEIRA,05585378805,M,14/06/1964,MG,Minas Novas,Superior
4,204560,https://dadosabertos.camara.leg.br/api/v2/depu...,ADOLFO VIANA DE CASTRO NETO,80123848504,M,02/02/1981,BA,Salvador,Superior


In [28]:
df_deputados.groupby('sexo', dropna=False)['id'].nunique(dropna=False)

sexo
F    103
M    476
Name: id, dtype: int64

In [29]:
df_deputados['id'].duplicated().sum()

23

In [30]:
# Verificar se existem IDs duplicados
duplicados = df_deputados[df_deputados.duplicated(subset=['id'], keep=False)]
print(duplicados)

         id                                                uri  \
21   220542  https://dadosabertos.camara.leg.br/api/v2/depu...   
22   220542  https://dadosabertos.camara.leg.br/api/v2/depu...   
29   226708  https://dadosabertos.camara.leg.br/api/v2/depu...   
88    74262  https://dadosabertos.camara.leg.br/api/v2/depu...   
89    74262  https://dadosabertos.camara.leg.br/api/v2/depu...   
104  204476  https://dadosabertos.camara.leg.br/api/v2/depu...   
105  204476  https://dadosabertos.camara.leg.br/api/v2/depu...   
171  226708  https://dadosabertos.camara.leg.br/api/v2/depu...   
176  204518  https://dadosabertos.camara.leg.br/api/v2/depu...   
177  204518  https://dadosabertos.camara.leg.br/api/v2/depu...   
186  227800  https://dadosabertos.camara.leg.br/api/v2/depu...   
187  227800  https://dadosabertos.camara.leg.br/api/v2/depu...   
188  227800  https://dadosabertos.camara.leg.br/api/v2/depu...   
222  227370  https://dadosabertos.camara.leg.br/api/v2/depu...   
223  22737

In [31]:
df_deputados['duplicado'] = df_deputados.duplicated(subset=['id'], keep=False)

In [32]:
df_duplicados = df_deputados[df_deputados['duplicado'] == True]
df_duplicados[['id', 'nomeCivil', 'sexo', 'ufNascimento']]

,id,nomeCivil,sexo,ufNascimento
21,220542,JOSÉ ALEXANDRE DOMINGUES GUIMARÃES,M,TO
22,220542,JOSÉ ALEXANDRE DOMINGUES GUIMARÃES,M,TO
29,226708,ALLAN QUADROS GARCÊS,M,PA
88,74262,CARLOS HENRIQUE FOCESI SAMPAIO,M,SP
89,74262,CARLOS HENRIQUE FOCESI SAMPAIO,M,SP
104,204476,JOÃO FRANCISCO INÁCIO BRAZÃO,M,RJ
105,204476,JOÃO FRANCISCO INÁCIO BRAZÃO,M,RJ
171,226708,ALLAN QUADROS GARCÊS,M,PA
176,204518,FREDERICO DE CASTRO ESCALEIRA,M,RJ
177,204518,FREDERICO DE CASTRO ESCALEIRA,M,RJ


In [33]:
print(df_deputados.shape)
# Remover duplicatas, mantendo a primeira ocorrência
df_deputados = df_deputados.drop_duplicates(subset=['id'])
print(df_deputados.shape)

(602, 10)
(579, 10)


In [34]:
df_deputados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 579 entries, 0 to 600
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   579 non-null    int64 
 1   uri                  579 non-null    object
 2   nomeCivil            579 non-null    object
 3   cpf                  579 non-null    object
 4   sexo                 579 non-null    object
 5   dataNascimento       579 non-null    object
 6   ufNascimento         579 non-null    object
 7   municipioNascimento  579 non-null    object
 8   escolaridade         579 non-null    object
 9   duplicado            579 non-null    bool  
dtypes: bool(1), int64(1), object(8)
memory usage: 45.8+ KB


In [35]:
print(df_deputados[df_deputados['sexo'] == 'F'].shape[0]) 

103


In [36]:
resultado = df_deputados[df_deputados['sexo'] == 'F']['ufNascimento'].value_counts(dropna=False)
resultado

ufNascimento
SP    16
MG    12
RJ    11
PR     8
RS     7
CE     6
BA     5
PA     5
SC     5
MA     4
GO     3
PE     3
AP     3
MT     2
RO     2
AC     2
SE     2
PB     2
ES     2
MS     1
TO     1
RN     1
Name: count, dtype: int64

In [37]:
df_deputados._is_view

False

In [38]:
df_deputados.loc[:, 'ano_nascimento'] = pd.to_datetime(df_deputados['dataNascimento'], dayfirst=True).dt.year

In [39]:
def determinar_geracao(ano_nascimento):
  if ano_nascimento >= 1997:
    return "Geração Z"
  elif 1981 <= ano_nascimento <= 1996:
    return "Millennials (Geração Y)"
  elif 1965 <= ano_nascimento <= 1980:
    return "Geração X"
  elif 1946 <= ano_nascimento <= 1964:
    return "Baby Boomers"
  else:
    return "Geração Silenciosa"

In [40]:
df_deputados.loc[:, 'geração'] = df_deputados['ano_nascimento'].apply(determinar_geracao)

In [41]:
df_deputados['geração'].value_counts()

geração
Geração X                  249
Baby Boomers               164
Millennials (Geração Y)    154
Geração Z                    6
Geração Silenciosa           6
Name: count, dtype: int64

In [42]:
df_deputados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 579 entries, 0 to 600
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   579 non-null    int64 
 1   uri                  579 non-null    object
 2   nomeCivil            579 non-null    object
 3   cpf                  579 non-null    object
 4   sexo                 579 non-null    object
 5   dataNascimento       579 non-null    object
 6   ufNascimento         579 non-null    object
 7   municipioNascimento  579 non-null    object
 8   escolaridade         579 non-null    object
 9   duplicado            579 non-null    bool  
 10  ano_nascimento       579 non-null    int32 
 11  geração              579 non-null    object
dtypes: bool(1), int32(1), int64(1), object(9)
memory usage: 52.6+ KB


In [43]:
from datetime import datetime

# Converter a coluna 'ano_nascimento' para inteiro, tratando possíveis NaNs
df_deputados.loc[:, 'ano_nascimento'] = pd.to_numeric(df_deputados['ano_nascimento'], errors='coerce').fillna(0).astype(int)

# Calcular a idade corretamente
df_deputados.loc[:, 'idade'] = datetime.now().year - df_deputados['ano_nascimento']

In [44]:
df_deputados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 579 entries, 0 to 600
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   579 non-null    int64 
 1   uri                  579 non-null    object
 2   nomeCivil            579 non-null    object
 3   cpf                  579 non-null    object
 4   sexo                 579 non-null    object
 5   dataNascimento       579 non-null    object
 6   ufNascimento         579 non-null    object
 7   municipioNascimento  579 non-null    object
 8   escolaridade         579 non-null    object
 9   duplicado            579 non-null    bool  
 10  ano_nascimento       579 non-null    int32 
 11  geração              579 non-null    object
 12  idade                579 non-null    int32 
dtypes: bool(1), int32(2), int64(1), object(9)
memory usage: 54.8+ KB


In [45]:
df_deputados.head()

,id,uri,nomeCivil,cpf,sexo,dataNascimento,ufNascimento,municipioNascimento,escolaridade,duplicado,ano_nascimento,geração,idade
0,220593,https://dadosabertos.camara.leg.br/api/v2/depu...,ABILIO JACQUES BRUNINI MOUMER,99770962104,M,31/01/1984,MT,Cuiabá,Superior,False,1984,Millennials (Geração Y),41
1,204379,https://dadosabertos.camara.leg.br/api/v2/depu...,ACÁCIO DA SILVA FAVACHO NETO,74287028287,M,28/09/1983,AP,Macapá,Superior,False,1983,Millennials (Geração Y),42
2,220714,https://dadosabertos.camara.leg.br/api/v2/depu...,ADAIL JOSÉ FIGUEIREDO PINHEIRO,77267796249,M,16/02/1992,AM,Manaus,Superior Incompleto,False,1992,Millennials (Geração Y),33
3,221328,https://dadosabertos.camara.leg.br/api/v2/depu...,ADILSON BARROSO OLIVEIRA,05585378805,M,14/06/1964,MG,Minas Novas,Superior,False,1964,Baby Boomers,61
4,204560,https://dadosabertos.camara.leg.br/api/v2/depu...,ADOLFO VIANA DE CASTRO NETO,80123848504,M,02/02/1981,BA,Salvador,Superior,False,1981,Millennials (Geração Y),44
